In [35]:
'''

Data Split
Use train_dataset and eval_dataset as train / test sets

'''
from torchvision.datasets import EMNIST
from torch.utils.data import ConcatDataset, Subset
from torchvision.transforms import ToTensor, Compose
import numpy as np

# the library imported by myself
import torch
import torch.nn as nn

import os
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from torch.optim import Adam, SGD # just choose which to use
from torchvision.utils import make_grid
import matplotlib.pyplot as plt



# For convenience, show image at index in dataset
def show_image(dataset, index):
  
  plt.imshow(dataset[index][0][0], cmap=plt.get_cmap('gray'))

def get_datasets(split='balanced', save=False):
  download_folder = './data'

  transform = Compose([ToTensor()])

  dataset = ConcatDataset([EMNIST(root=download_folder, split=split, download=True, train=False, transform=transform),
                           EMNIST(root=download_folder, split=split, download=True, train=True, transform=transform)])

  # save already = false, the program will not go into the below part, I just leave it.
  if save:
    random_seed = 4211 # do not change
    n_samples = len(dataset)
    eval_size = 0.2
    indices = list(range(n_samples))
    split = int(np.floor(eval_size * n_samples))

    np.random.seed(random_seed)
    np.random.shuffle(indices)

    train_indices, eval_indices = indices[split:], indices[:split]

    # cut to half
    train_indices = train_indices[:len(train_indices)//2]
    eval_indices = eval_indices[:len(eval_indices)//2]

    np.savez('train_test_split.npz', train=train_indices, test=eval_indices)

  # This is what really happen
  # load train test split indices
  else:
    with np.load('./train_test_split.npz') as f:
      train_indices = f['train']
      eval_indices = f['test']

  train_dataset = Subset(dataset, indices=train_indices)
  eval_dataset = Subset(dataset, indices=eval_indices)

  return train_dataset, eval_dataset

# TODO
# 1. build your own CNN classifier with the given structure. DO NOT COPY OR USE ANY TRICK
# 2. load pretrained encoder from 'pretrained_encoder.pt' and build a CNN classifier on top of the encoder
# 3. load pretrained encoder from 'pretrained_encoder.pt' and build a Convolutional Autoencoder on top of the encoder (just need to implement decoder)
# *** Note that all the above tasks include implementation, training, analyzing, and reporting

# example main code
# each img has size (1, 28, 28) and each label is in {0, ..., 46}, a total of 47 classes


######################################################################################################################################
# For the task 3.2.1 : constructing the CNN model required
class Cnn_Scratch_Classifier(nn.Module):
  def __init__(self, n_hidden):
    super(Cnn_Scratch_Classifier, self).__init__()

    # in_data size: (batch_size, 1, 28, 28)
    self.cnn_layers = nn.Sequential(
      # conv1_out size: (batch_size, 4, 26, 26)
      nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding=0),
      nn.ReLU(),
      # conv2_out size: (batch_size, 8, 12, 12)
      nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2, padding=0),
      nn.ReLU(),
      # conv3_out size: (batch_size, 16, 5, 5)
      nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=0),
      nn.ReLU(),
      # MaxPool2d_out size: (batch_size, 16, 3, 3)
      nn.MaxPool2d(kernel_size=3, stride=1, padding=0),
      # conv4_out size: (batch_size, 32, 1, 1)
      nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
      # As required, just after the last convolution layer, logistic function should be used
      nn.Sigmoid()
    )

    # linear layers transforms flattened image features into logits before the softmax layer
    self.linear = nn.Sequential(
      nn.Linear(32, n_hidden),
      nn.ReLU(),
      nn.Linear(n_hidden, 47) # there are 47 predicted_labels
    )

    self.softmax = nn.Softmax(dim=1)
    self.loss_fn = nn.CrossEntropyLoss(reduction='sum') # will be divided by batch size

  def forward(self, in_data):
    img_features = self.cnn_layers(in_data).view(in_data.size(0), 32) # in_data.size(0) == batch_size
    logits = self.linear(img_features)
    return logits

  def loss(self, logits, labels):
    #preds = self.softmax(logits) # size (batch_size, 10)
    return self.loss_fn(logits, labels) / logits.size(0) # divided by batch_size


  def top1_accuracy(self, logits, labels):
    # get argmax of logits along dim=1 (this is equivalent to argmax of predicted probabilites)
    predicted_labels = torch.argmax(logits, dim=1, keepdim=False)
    n_corrects = float(predicted_labels.eq(labels).sum(0)) # sum up all the correct predictions
    return int(n_corrects / logits.size(0) * 100) # in percentage

  def top3_accuracy(self, logits, labels):
    n_corrects = 0
    # copy logits for implementation without chage original value
    logits_copy = logits.clone().detach()
    # get argmax of logits along dim=1 (this is equivalent to argmax of predicted probabilites)
    max1 = torch.argmax(logits_copy, dim=1)
    n_corrects = n_corrects + max1.eq(labels).sum(0)
    for i in range(max1.shape[0]):
        logits_copy[i][max1[i]] = -99999

    max2 = torch.argmax(logits_copy, dim=1)
    n_corrects = n_corrects + max2.eq(labels).sum(0)
    for i in range(max2.shape[0]):
        logits_copy[i][max2[i]] = -99999

    max3 = torch.argmax(logits_copy, dim=1)
    n_corrects = float(n_corrects + max3.eq(labels).sum(0))
    for i in range(max3.shape[0]):
        logits_copy[i][max3[i]] = -99999

    return int((n_corrects / logits.size(0)) * 100) # in percentage

######################################################################################################################################

# For the task 3.2.2: Learning from Pretrained Encoder Weights


# just load the pt file, then do the same linear operation as Cnn_Scratch_Classifier

class Cnn_Pretrained_Classifier(nn.Module):
 def __init__(self, n_hidden):
   
   super(Cnn_Pretrained_Classifier, self).__init__()
   
   self.cnn_layers = torch.load('pretrained_encoder.pt', map_location = 'cpu')['model']

   # linear layers transforms flattened image features into logits before the softmax layer
   self.linear = nn.Sequential(
     nn.Linear(32, n_hidden),
     nn.ReLU(),
     nn.Linear(n_hidden, 47) # there are 47 predicted_labels
   )

   self.softmax = nn.Softmax(dim=1)
   self.loss_fn = nn.CrossEntropyLoss(reduction='sum') # will be divided by batch size

 def forward(self, in_data):
    img_features = self.cnn_layers(in_data).view(in_data.size(0), 32) # in_data.size(0) == batch_size
    logits = self.linear(img_features)
    return logits

 def loss(self, logits, labels):
    #preds = self.softmax(logits) # size (batch_size, 10)
    return self.loss_fn(logits, labels) / logits.size(0) # divided by batch_size


 def top1_accuracy(self, logits, labels):
   # get argmax of logits along dim=1 (this is equivalent to argmax of predicted probabilites)
   predicted_labels = torch.argmax(logits, dim=1, keepdim=False) # size (batch_size,)
   n_corrects = float(predicted_labels.eq(labels).sum(0)) # sum up all the correct predictions
   return int(n_corrects / logits.size(0) * 100) # in percentage


 def top3_accuracy(self, logits, labels):
    n_corrects = 0
    # copy logits for implementation without chage original value
    logits_copy = logits.clone().detach()
    # get argmax of logits along dim=1 (this is equivalent to argmax of predicted probabilites)
    max1 = torch.argmax(logits_copy, dim=1)
    n_corrects = n_corrects + max1.eq(labels).sum(0)
    for i in range(max1.shape[0]):
        logits_copy[i][max1[i]] = -99999

    max2 = torch.argmax(logits_copy, dim=1)
    n_corrects = n_corrects + max2.eq(labels).sum(0)
    for i in range(max2.shape[0]):
        logits_copy[i][max2[i]] = -99999

    max3 = torch.argmax(logits_copy, dim=1)
    n_corrects = float(n_corrects + max3.eq(labels).sum(0))
    for i in range(max3.shape[0]):
        logits_copy[i][max3[i]] = -99999

    return int(n_corrects / logits.size(0) * 100 )# in percentage

######################################################################################################################################3

# Train function
# Notice that the eval here is not the test set, but the validation set.

from torchvision.utils import make_grid

def train(model, loaders, optimizer, writer, n_epochs, ckpt_path, device='cpu'):
  def run_epoch(train_or_eval):
    epoch_loss = 0.
    epoch_acc_1 = 0.
    epoch_acc_3 = 0.

    for i, batch in enumerate(loaders[train_or_eval], 1):
      in_data, labels = batch
      in_data, labels = in_data.to(device), labels.to(device)

      if train_or_eval == 'train':
        optimizer.zero_grad()

      logits = model(in_data)
      
      batch_loss = model.loss(logits, labels)
      batch_acc_1 = model.top1_accuracy(logits, labels)
      batch_acc_3 = model.top3_accuracy(logits, labels)

      epoch_loss += batch_loss
      epoch_acc_1 += batch_acc_1
      epoch_acc_3 += batch_acc_3

      if train_or_eval == 'train':
        batch_loss.backward()
        optimizer.step()

    epoch_loss /= i
    epoch_acc_1 /= i
    epoch_acc_3 /= i
    print('loss: %s    top1 acc: %s     top3 acc: %s' %(epoch_loss,epoch_acc_1,epoch_acc_3))
    print()

    losses[train_or_eval] = epoch_loss
    accs_1[train_or_eval] = epoch_acc_1
    accs_3[train_or_eval] = epoch_acc_3

    if writer is None:
      print('epoch %d %s loss %.4f acc %.4f' % (epoch, train_or_eval, epoch_loss, epoch_acc))
    elif train_or_eval == 'eval':
      writer.add_scalars('%s_loss' % model.__class__.__name__,
                         tag_scalar_dict={'train': losses['train'],
                                          'eval': losses['eval']},
                         global_step=epoch)

      writer.add_scalars('%s_top1_accuracy' % model.__class__.__name__,  
                         tag_scalar_dict={'train': accs_1['train'],
                                          'eval': accs_1['eval']},
                         global_step=epoch)

      writer.add_scalars('%s_top3_accuracy' % model.__class__.__name__,  # CnnClassifier or FcClassifier
                         tag_scalar_dict={'train': accs_3['train'],
                                          'eval': accs_3['eval']},
                         global_step=epoch)

      # For instructional purpose, add images here, just the last in_data
      #if epoch % 10 == 0:
      #  if len(in_data.size()) == 2: # when it is flattened, reshape it
      #    in_data = in_data.view(-1, 1, 28, 28)

      #  img_grid = make_grid(in_data.to('cpu'))
      #  writer.add_image('%s/eval_input' % model.__class__.__name__, img_grid, epoch)

  # main statements
  losses = dict()
  accs_1 = dict()
  accs_3 = dict()

  for epoch in range(1, n_epochs+1):
    print('Epoch %s :' %epoch)
    print('Train:')
    run_epoch('train')
    print('Eval:')
    run_epoch('eval')

    # For instructional purpose, show how to save checkpoints
    if ckpt_path is not None:
      torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'losses': losses,
        'accs_1': accs_1,
        'accs_3': accs_3
      }, '%s/%d.pt' % (ckpt_path, epoch))

######################################################################################################################################

# this train function is for training the CNN with the optimal parameter set, which do not have holdout
def train_for_test(model, loaders, optimizer, writer, n_epochs, ckpt_path, device='cpu'):
  def run_epoch(train_or_eval):
    epoch_loss = 0.
    epoch_acc_1 = 0.
    epoch_acc_3 = 0.

    for i, batch in enumerate(loaders[train_or_eval], 1):
      in_data, labels = batch
      in_data, labels = in_data.to(device), labels.to(device)

      if train_or_eval == 'train':
        optimizer.zero_grad()

      logits = model(in_data)
      
      batch_loss = model.loss(logits, labels)
      batch_acc_1 = model.top1_accuracy(logits, labels)
      batch_acc_3 = model.top3_accuracy(logits, labels)

      epoch_loss += batch_loss
      epoch_acc_1 += batch_acc_1
      epoch_acc_3 += batch_acc_3

      if train_or_eval == 'train':
        batch_loss.backward()
        optimizer.step()

    epoch_loss /= i
    epoch_acc_1 /= i
    epoch_acc_3 /= i
    print('top1 acc: %s     top3 acc: %s' %(epoch_acc_1,epoch_acc_3))
    print()

    losses[train_or_eval] = epoch_loss
    accs_1[train_or_eval] = epoch_acc_1
    accs_3[train_or_eval] = epoch_acc_3

    if writer is None:
      print('epoch %d %s loss %.4f acc %.4f' % (epoch, train_or_eval, epoch_loss, epoch_acc))
    elif train_or_eval == 'eval':
      writer.add_scalars('%s_loss' % model.__class__.__name__,
                         tag_scalar_dict={'train': losses['train'],
                                          'eval': losses['eval']},
                         global_step=epoch)

      writer.add_scalars('%s_top1_accuracy' % model.__class__.__name__,  
                         tag_scalar_dict={'train': accs_1['train'],
                                          'eval': accs_1['eval']},
                         global_step=epoch)

      writer.add_scalars('%s_top3_accuracy' % model.__class__.__name__,  # CnnClassifier or FcClassifier
                         tag_scalar_dict={'train': accs_3['train'],
                                          'eval': accs_3['eval']},
                         global_step=epoch)

      # For instructional purpose, add images here, just the last in_data
      #if epoch % 10 == 0:
      #  if len(in_data.size()) == 2: # when it is flattened, reshape it
      #    in_data = in_data.view(-1, 1, 28, 28)

      #  img_grid = make_grid(in_data.to('cpu'))
      #  writer.add_image('%s/eval_input' % model.__class__.__name__, img_grid, epoch)

  # main statements
  losses = dict()
  accs_1 = dict()
  accs_3 = dict()

  for epoch in range(1, n_epochs+1):
    print('Epoch %s :' %epoch)
    print('Train:')
    run_epoch('train')
    print('Test:')
    run_epoch('eval')

    # For instructional purpose, show how to save checkpoints
    if ckpt_path is not None:
      torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'losses': losses,
        'accs_1': accs_1,
        'accs_3': accs_3
      }, '%s/%d.pt' % (ckpt_path, epoch))


###########################################################################################################333


class Cnn_Reconstructor(nn.Module):
 def __init__(self):
   super(Cnn_Reconstructor, self).__init__()

   self.cnn_layers = torch.load('pretrained_encoder.pt', map_location = 'cpu')['model']
   
   # linear layers transforms flattened image features into logits before the softmax layer
   self.decoder = nn.Sequential(
     
     nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=0),
     nn.ReLU(),

     nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=0),
     nn.ReLU(),

     nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=3, stride=2, padding=0),
     nn.ReLU(),

     nn.ConvTranspose2d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=0),
     nn.ReLU(),

     nn.ConvTranspose2d(in_channels=4, out_channels=1, kernel_size=4, stride=2, padding=0),

     # As required, just after the last convolution layer, logistic function should be used
     nn.Sigmoid()
   )

   self.softmax = nn.Softmax(dim=1)
   # As required, here i need to use the MSE loss as the loss function
   self.loss_fn = nn.MSELoss(reduction='sum') # will be divided by batch size

 def forward(self, in_data):
   img_features = self.cnn_layers(in_data).view(in_data.size(0), 32,1,1) # in_data.size(0) == batch_size
   logits = self.decoder(img_features)
   return logits

 def loss(self, logits, labels):
   #preds = self.softmax(logits) # size (batch_size, 10)
   return self.loss_fn(logits, labels) / logits.size(0) # divided by batch_size


 #def top1_accuracy(self, logits, labels):
   # get argmax of logits along dim=1 (this is equivalent to argmax of predicted probabilites)
   #predicted_labels = torch.argmax(logits, dim=1, keepdim=False) # size (batch_size,)
   #n_corrects = predicted_labels.eq(labels).sum(0) # sum up all the correct predictions
   #return n_corrects / logits.size(0) * 100 # in percentage

 #def top3_accuracy(self, logits, labels):
    #n_corrects = 0
     # copy logits for implementation without chage original value
    #logits_copy = logits.clone().detach()
     # get argmax of logits along dim=1 (this is equivalent to argmax of predicted probabilites)
    #max1 = torch.argmax(logits_copy, dim=1)
    #n_corrects = n_corrects + max1.eq(labels).sum(0)
    #for i in range(max1.shape[0]):
    #    logits_copy[i][max1[i]] = -99999

    #max2 = torch.argmax(logits_copy, dim=1)
    #n_corrects = n_corrects + max2.eq(labels).sum(0)
    #for i in range(max2.shape[0]):
    #    logits_copy[i][max2[i]] = -99999

    #max3 = torch.argmax(logits_copy, dim=1)
    #n_corrects = n_corrects + max3.eq(labels).sum(0)
    #for i in range(max3.shape[0]):
    #    logits_copy[i][max3[i]] = -99999

    #return n_corrects / logits.size(0) * 100 # in percentage
    
#####################################################################################################################
# To train the CNN reconstructor, I wrote a new train function
def train_reconstructor(model, loaders, optimizer, writer, n_epochs, ckpt_path, device='cpu'):
  def run_epoch(train_or_eval):
    epoch_loss = 0.

    for i, batch in enumerate(loaders[train_or_eval], 1):
      in_data, labels = batch
      in_data, labels = in_data.to(device), labels.to(device)

      if train_or_eval == 'train':
        optimizer.zero_grad()

      logits = model(in_data)
      
      batch_loss = model.loss(logits, in_data)
      
     
      epoch_loss += batch_loss
        
      if train_or_eval == 'train':
        batch_loss.backward()
        optimizer.step()
      if train_or_eval == 'eval':  
        if len(logits.size()) == 2: # when it is flattened, reshape it
          logits = logits.view(-1, 1, 28, 28)

        img_grid = make_grid(logits.to('cpu'))
        writer.add_image('%s/eval_input' % model.__class__.__name__, img_grid, epoch)

    epoch_loss /= i
    print('loss: %s' %epoch_loss)
    print()

    losses[train_or_eval] = epoch_loss

    if writer is None:
      print('epoch %d %s loss %.4f acc %.4f' % (epoch, train_or_eval, epoch_loss, epoch_acc))
    elif train_or_eval == 'eval':
      writer.add_scalars('%s_loss' % model.__class__.__name__,
                         tag_scalar_dict={'train': losses['train'],
                                          'eval': losses['eval']},
                         global_step=epoch)


     
      
    #  if len(logits.size()) == 2: # when it is flattened, reshape it
    #    logits = logits.view(-1, 1, 28, 28)

   #   img_grid = make_grid(logits.to('cpu'))
   #   writer.add_image('%s/eval_input' % model.__class__.__name__, img_grid, epoch)

  # main statements
  losses = dict()
 

  for epoch in range(1, n_epochs+1):
    print('Epoch %s :' %epoch)
    print('Train:')
    run_epoch('train')
    print('Eval:')
    run_epoch('eval')

    # For instructional purpose, show how to save checkpoints
    if ckpt_path is not None:
      torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'losses': losses
      }, '%s/%d.pt' % (ckpt_path, epoch))



    
########################################################################################################################   






In [36]:
# TODO
# 1. build your own CNN classifier with the given structure. DO NOT COPY OR USE ANY TRICK


## Main function for implementing training
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from torch.optim import Adam, SGD # just choose which to use


def cnn_scratch_main(n_hidden,optim,learning_rate):
  
  gpu = -1 # default
  #lr = args.lr
  lr = learning_rate
  #batch_size = args.batch
  batch_size = 32 # default
  #ckpt_path = args.ckpt
  ckpt_path = './ckpt/cnn'
  #n_epochs = args.epoch
  n_epochs = 10 # default
  #opt_str = args.optim
  opt_str = optim

  ckpt_path = '%s/%s' % (ckpt_path, opt_str)

  if ckpt_path is not None:
    if not(os.path.exists(ckpt_path)):
      os.makedirs(ckpt_path)

  if gpu == -1:
    DEVICE = 'cpu'
  elif torch.cuda.is_available():
    DEVICE = gpu

  #model = CnnClassifier(n_hidden).to(DEVICE)
  # For the task 3.2.1, i need the Cnn_Scratch_Classifier model
  model = Cnn_Scratch_Classifier(n_hidden)

  train_dataset, test_dataset = get_datasets()

  # Here I need to split the train_dataset in to train and eval with the ratio 4:1. I determine to set the last 20% as validation  dataset

  n_samples = len(train_dataset)
  eval_start = 0.8
  split = int(np.floor(eval_start * n_samples))
  indices = list(range(n_samples))
  train_indices, validation_indices = indices[:split], indices[split:]

  dataloaders = {
    'train':  DataLoader(Subset(train_dataset, indices=train_indices),batch_size=batch_size, drop_last=False, shuffle=True),
    'eval':   DataLoader(Subset(train_dataset, indices=validation_indices),batch_size=batch_size, drop_last=False, shuffle=True)
  }

  if opt_str == 'adam':
    opt_class = Adam
  elif opt_str == 'sgd':
    opt_class = SGD

  optimizer = opt_class(model.parameters(), lr=lr)
  writer = SummaryWriter('./logs/cnn_scratch/%s_%s_%s' % (opt_str,n_hidden,lr))

  train(model, dataloaders, optimizer, writer, n_epochs, ckpt_path, DEVICE)









#if __name__=='__main__':

  # Task 3.2.1 : Use my own CNN classifier
  #cnn_scratch_main(32,'adam',0.001)
 # cnn_scratch_main(64,'adam',0.001)
 # cnn_scratch_main(32,'sgd',0.1)
 # cnn_scratch_main(64,'sgd',0.1)
 # cnn_scratch_main(32,'sgd',0.01)
 # cnn_scratch_main(64,'sgd',0.01)





In [37]:
cnn_scratch_main(32,'adam',0.001)

Epoch 1 :
Train:


KeyboardInterrupt: 

In [63]:
cnn_scratch_main(64,'adam',0.001)


Epoch 1 :
Train:
top1 acc: 27.481003039513677     top3 acc: 49.55775075987842

Eval:
top1 acc: 48.79331306990881     top3 acc: 79.01519756838906

Epoch 2 :
Train:
top1 acc: 56.651215805471125     top3 acc: 83.51215805471125

Eval:
top1 acc: 60.77811550151976     top3 acc: 87.04255319148936

Epoch 3 :
Train:
top1 acc: 64.2530395136778     top3 acc: 88.2629179331307

Eval:
top1 acc: 66.40121580547113     top3 acc: 89.86626139817629

Epoch 4 :
Train:
top1 acc: 68.40881458966565     top3 acc: 90.42629179331307

Eval:
top1 acc: 68.09118541033435     top3 acc: 90.60182370820668

Epoch 5 :
Train:
top1 acc: 71.34042553191489     top3 acc: 91.79331306990882

Eval:
top1 acc: 70.1306990881459     top3 acc: 91.84802431610942

Epoch 6 :
Train:
top1 acc: 73.15425531914893     top3 acc: 92.71048632218844

Eval:
top1 acc: 72.71428571428571     top3 acc: 93.0

Epoch 7 :
Train:
top1 acc: 74.76367781155015     top3 acc: 93.31914893617021

Eval:
top1 acc: 73.04559270516717     top3 acc: 93.31306990881458


In [64]:
cnn_scratch_main(32,'sgd',0.1)

Epoch 1 :
Train:
top1 acc: 2.069908814589666     top3 acc: 6.2803951367781155

Eval:
top1 acc: 2.024316109422492     top3 acc: 6.291793313069909

Epoch 2 :
Train:
top1 acc: 2.033434650455927     top3 acc: 6.114741641337386

Eval:
top1 acc: 1.750759878419453     top3 acc: 5.7477203647416415

Epoch 3 :
Train:
top1 acc: 2.040273556231003     top3 acc: 6.235562310030395

Eval:
top1 acc: 1.750759878419453     top3 acc: 5.890577507598784

Epoch 4 :
Train:
top1 acc: 2.0972644376899696     top3 acc: 6.330547112462006

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.218844984802431

Epoch 5 :
Train:
top1 acc: 1.987841945288754     top3 acc: 6.069148936170213

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.382978723404255

Epoch 6 :
Train:
top1 acc: 2.1861702127659575     top3 acc: 6.28419452887538

Eval:
top1 acc: 2.069908814589666     top3 acc: 6.2097264437689965

Epoch 7 :
Train:
top1 acc: 2.1268996960486324     top3 acc: 6.3373860182370825

Eval:
top1 acc: 2.115501519756839     top3 acc

In [65]:
cnn_scratch_main(64,'sgd',0.1)

Epoch 1 :
Train:
top1 acc: 2.1314589665653494     top3 acc: 6.198328267477204

Eval:
top1 acc: 1.9148936170212767     top3 acc: 5.735562310030395

Epoch 2 :
Train:
top1 acc: 1.9582066869300911     top3 acc: 6.189969604863222

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.2006079027355625

Epoch 3 :
Train:
top1 acc: 2.088145896656535     top3 acc: 6.354863221884498

Eval:
top1 acc: 2.270516717325228     top3 acc: 6.419452887537994

Epoch 4 :
Train:
top1 acc: 2.2454407294832825     top3 acc: 6.370060790273556

Eval:
top1 acc: 2.270516717325228     top3 acc: 6.382978723404255

Epoch 5 :
Train:
top1 acc: 2.188449848024316     top3 acc: 6.287993920972644

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.13677811550152

Epoch 6 :
Train:
top1 acc: 2.154255319148936     top3 acc: 6.389817629179332

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.1580547112462005

Epoch 7 :
Train:
top1 acc: 2.053951367781155     top3 acc: 6.122340425531915

Eval:
top1 acc: 2.115501519756839     top3 acc:

In [67]:
cnn_scratch_main(32,'sgd',0.01)

Epoch 1 :
Train:
top1 acc: 2.012917933130699     top3 acc: 6.026595744680851

Eval:
top1 acc: 2.0516717325227964     top3 acc: 6.37386018237082

Epoch 2 :
Train:
top1 acc: 2.167933130699088     top3 acc: 6.137537993920972

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.2948328267477205

Epoch 3 :
Train:
top1 acc: 2.236322188449848     top3 acc: 6.446808510638298

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.237082066869301

Epoch 4 :
Train:
top1 acc: 2.236322188449848     top3 acc: 6.429331306990881

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.13677811550152

Epoch 5 :
Train:
top1 acc: 2.236322188449848     top3 acc: 6.466565349544073

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.13677811550152

Epoch 6 :
Train:
top1 acc: 2.236322188449848     top3 acc: 6.48404255319149

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.139817629179332

Epoch 7 :
Train:
top1 acc: 2.236322188449848     top3 acc: 6.47112462006079

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.13677

In [68]:
cnn_scratch_main(64,'sgd',0.01)

Epoch 1 :
Train:
top1 acc: 2.0357142857142856     top3 acc: 6.018237082066869

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.4316109422492405

Epoch 2 :
Train:
top1 acc: 2.197568389057751     top3 acc: 6.25531914893617

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.13677811550152

Epoch 3 :
Train:
top1 acc: 2.229483282674772     top3 acc: 6.37917933130699

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.13677811550152

Epoch 4 :
Train:
top1 acc: 2.202127659574468     top3 acc: 6.506079027355623

Eval:
top1 acc: 2.8996960486322187     top3 acc: 6.382978723404255

Epoch 5 :
Train:
top1 acc: 2.1702127659574466     top3 acc: 6.39209726443769

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.13677811550152

Epoch 6 :
Train:
top1 acc: 2.227203647416413     top3 acc: 6.326747720364741

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.264437689969605

Epoch 7 :
Train:
top1 acc: 2.1633738601823707     top3 acc: 6.418693009118541

Eval:
top1 acc: 2.115501519756839     top3 acc: 6.227

In [29]:
# From the result I got, i would choose the adam 64 0.001 for the test
# Before that i build a test function


def cnn_scratch_test(n_hidden,optim,learning_rate):
  gpu = -1 # default
  
  lr = learning_rate
  
  batch_size = 32 # default
  #ckpt_path = args.ckpt
  ckpt_path = './ckpt/cnn'
  
  n_epochs = 20 # default
 
  opt_str = optim

  ckpt_path = '%s/%s' % (ckpt_path, opt_str)

  if ckpt_path is not None:
    if not(os.path.exists(ckpt_path)):
      os.makedirs(ckpt_path)

  if gpu == -1:
    DEVICE = 'cpu'
  elif torch.cuda.is_available():
    DEVICE = gpu

  #model = CnnClassifier(n_hidden).to(DEVICE)
  # For the task 3.2.1, i need the Cnn_Scratch_Classifier model
  model = Cnn_Scratch_Classifier(n_hidden)

  train_dataset, test_dataset = get_datasets()

  # Here I need to split the train_dataset in to train and eval with the ratio 4:1. I determine to set the last 20% as validation  dataset

  #n_samples = len(train_dataset)
  #eval_start = 0.8
  #split = int(np.floor(eval_start * n_samples))
  #indices = list(range(n_samples))
  #train_indices, validation_indices = indices[:split], indices[split:]

  dataloaders = {
    'train':  DataLoader(train_dataset,batch_size=batch_size, drop_last=False, shuffle=True),
    'eval':   DataLoader(test_dataset,batch_size=batch_size, drop_last=False, shuffle=True)
  }

  if opt_str == 'adam':
    opt_class = Adam
  elif opt_str == 'sgd':
    opt_class = SGD

  optimizer = opt_class(model.parameters(), lr=lr)
  writer = SummaryWriter('./logs/cnn_scratch/test_phase_%s_%s_%s' % (opt_str,n_hidden,lr))

  train_for_test(model, dataloaders, optimizer, writer, n_epochs, ckpt_path, DEVICE)

  ########################################
  #the above part is the same as the train_main function
  # Now i am gonna to use the trained model to do the test_dataset
  loss =0.
  acc_1= 0.
  acc_3= 0. 
  dataloaders_test = DataLoader(test_dataset,batch_size=batch_size, drop_last=False, shuffle=True)  
  
  for i, batch in enumerate(dataloaders_test, 1):
      in_data, labels = batch 
      logits = model(in_data)
      batch_loss = model.loss(logits, labels)
      batch_acc_1 = model.top1_accuracy(logits, labels)
      batch_acc_3 = model.top3_accuracy(logits, labels)

      loss +=  batch_loss
      acc_1 += batch_acc_1
      acc_3 += batch_acc_3
    
  loss /= i
  acc_1 /= i
  acc_3 /= i
    
    
    
    
  #in_data, labels = test_dataset
  #logits = model(in_data)
  #loss = model.loss(logits, labels)
  #acc_1 = model.top1_accuracy(logits, labels)
  #acc_3 = model.top3_accuracy(logits, labels)

 ###########
  print('Test Phase:')
  print('loss: %.4f acc1: %.4f acc3: %.4f' % (loss,acc_1,acc_3))












In [30]:
cnn_scratch_test(64,'adam',0.001)

Epoch 1 :
Train:


KeyboardInterrupt: 

In [31]:
def cnn_pretrained_main(n_hidden,optim,learning_rate):

  gpu = -1 # default
  #lr = args.lr
  lr = learning_rate
  #batch_size = args.batch
  batch_size = 32 # default
  #ckpt_path = args.ckpt
  ckpt_path = './ckpt/cnn'
  #n_epochs = args.epoch
  n_epochs = 10# default
  #opt_str = args.optim
  opt_str = optim

  ckpt_path = '%s/%s' % (ckpt_path, opt_str)

  if ckpt_path is not None:
    if not(os.path.exists(ckpt_path)):
      os.makedirs(ckpt_path)

  if gpu == -1:
    DEVICE = 'cpu'
  elif torch.cuda.is_available():
    DEVICE = gpu

  #model = CnnClassifier(n_hidden).to(DEVICE)
  # For the task 3.2.1, i need the Cnn_Scratch_Classifier model
  model = Cnn_Pretrained_Classifier(n_hidden)

  train_dataset, test_dataset = get_datasets()

  # Here I need to split the train_dataset in to train and eval with the ratio 4:1. I determine to set the last 20% as validation  dataset

  n_samples = len(train_dataset)
  eval_start = 0.8
  split = int(np.floor(eval_start * n_samples))
  indices = list(range(n_samples))
  train_indices, validation_indices = indices[:split], indices[split:]

  dataloaders = {
    'train':  DataLoader(Subset(train_dataset, indices=train_indices),batch_size=batch_size, drop_last=False, shuffle=True),
    'eval':   DataLoader(Subset(train_dataset, indices=validation_indices),batch_size=batch_size, drop_last=False, shuffle=True)
  }

  if opt_str == 'adam':
    opt_class = Adam
  elif opt_str == 'sgd':
    opt_class = SGD

  optimizer = opt_class(model.parameters(), lr=lr)
  writer = SummaryWriter('./logs/cnn_pretrained/%s_%s_%s' % (opt_str,n_hidden,lr))

  train(model, dataloaders, optimizer, writer, n_epochs, ckpt_path, DEVICE)


In [32]:
cnn_pretrained_main(32,'adam',0.001)

Epoch 1 :
Train:


KeyboardInterrupt: 

In [104]:
cnn_pretrained_main(64,'adam',0.001)

Epoch 1 :
Train:
loss: tensor(2.6514, grad_fn=<DivBackward0>)    top1 acc: 36.57446808510638     top3 acc: 61.2887537993921

Eval:
loss: tensor(1.7998, grad_fn=<DivBackward0>)    top1 acc: 52.525835866261396     top3 acc: 78.51671732522796

Epoch 2 :
Train:
loss: tensor(1.5708, grad_fn=<DivBackward0>)    top1 acc: 56.287993920972646     top3 acc: 81.18465045592706

Eval:
loss: tensor(1.4274, grad_fn=<DivBackward0>)    top1 acc: 58.17325227963526     top3 acc: 83.40121580547113

Epoch 3 :
Train:
loss: tensor(1.3497, grad_fn=<DivBackward0>)    top1 acc: 60.33206686930091     top3 acc: 84.11398176291793

Eval:
loss: tensor(1.3077, grad_fn=<DivBackward0>)    top1 acc: 60.27355623100304     top3 acc: 84.89057750759878

Epoch 4 :
Train:
loss: tensor(1.2589, grad_fn=<DivBackward0>)    top1 acc: 62.34270516717325     top3 acc: 85.25607902735563

Eval:
loss: tensor(1.2352, grad_fn=<DivBackward0>)    top1 acc: 62.82066869300912     top3 acc: 86.03647416413374

Epoch 5 :
Train:
loss: tensor(1.208

In [33]:
cnn_pretrained_main(32,'sgd',0.1)

Epoch 1 :
Train:


KeyboardInterrupt: 

In [100]:
cnn_pretrained_main(64,'sgd',0.1)

Epoch 1 :
Train:
loss: tensor(3.1381, grad_fn=<DivBackward0>)    top1 acc: 25.96352583586626     top3 acc: 47.72796352583587

Eval:
loss: tensor(2.1194, grad_fn=<DivBackward0>)    top1 acc: 42.650455927051674     top3 acc: 72.10030395136778

Epoch 2 :
Train:
loss: tensor(1.6818, grad_fn=<DivBackward0>)    top1 acc: 52.952887537993924     top3 acc: 78.98176291793312

Eval:
loss: tensor(1.4444, grad_fn=<DivBackward0>)    top1 acc: 57.27051671732523     top3 acc: 82.74772036474164

Epoch 3 :
Train:
loss: tensor(1.3517, grad_fn=<DivBackward0>)    top1 acc: 59.659574468085104     top3 acc: 83.77507598784194

Eval:
loss: tensor(1.2924, grad_fn=<DivBackward0>)    top1 acc: 60.99696048632219     top3 acc: 85.15197568389058

Epoch 4 :
Train:
loss: tensor(1.2414, grad_fn=<DivBackward0>)    top1 acc: 62.53191489361702     top3 acc: 85.52127659574468

Eval:
loss: tensor(1.2033, grad_fn=<DivBackward0>)    top1 acc: 63.32522796352583     top3 acc: 86.46808510638297

Epoch 5 :
Train:
loss: tensor(1.1

In [101]:
cnn_pretrained_main(32,'sgd',0.01)

Epoch 1 :
Train:
loss: tensor(3.8424, grad_fn=<DivBackward0>)    top1 acc: 3.7066869300911853     top3 acc: 7.8465045592705165

Eval:
loss: tensor(3.8254, grad_fn=<DivBackward0>)    top1 acc: 5.079027355623101     top3 acc: 10.635258358662615

Epoch 2 :
Train:
loss: tensor(3.7974, grad_fn=<DivBackward0>)    top1 acc: 7.170212765957447     top3 acc: 16.49544072948328

Eval:
loss: tensor(3.7677, grad_fn=<DivBackward0>)    top1 acc: 9.349544072948328     top3 acc: 21.35562310030395

Epoch 3 :
Train:
loss: tensor(3.7213, grad_fn=<DivBackward0>)    top1 acc: 12.004559270516717     top3 acc: 27.475683890577507

Eval:
loss: tensor(3.6686, grad_fn=<DivBackward0>)    top1 acc: 14.46808510638298     top3 acc: 31.94224924012158

Epoch 4 :
Train:
loss: tensor(3.5920, grad_fn=<DivBackward0>)    top1 acc: 17.404255319148938     top3 acc: 36.62537993920973

Eval:
loss: tensor(3.5075, grad_fn=<DivBackward0>)    top1 acc: 19.811550151975684     top3 acc: 41.08206686930091

Epoch 5 :
Train:
loss: tensor

In [103]:
cnn_pretrained_main(64,'sgd',0.01)

Epoch 1 :
Train:
loss: tensor(3.8346, grad_fn=<DivBackward0>)    top1 acc: 4.72644376899696     top3 acc: 10.544072948328267

Eval:
loss: tensor(3.8027, grad_fn=<DivBackward0>)    top1 acc: 7.267477203647417     top3 acc: 18.580547112462007

Epoch 2 :
Train:
loss: tensor(3.7660, grad_fn=<DivBackward0>)    top1 acc: 10.24696048632219     top3 acc: 26.8031914893617

Eval:
loss: tensor(3.7229, grad_fn=<DivBackward0>)    top1 acc: 14.683890577507599     top3 acc: 37.02431610942249

Epoch 3 :
Train:
loss: tensor(3.6594, grad_fn=<DivBackward0>)    top1 acc: 19.75987841945289     top3 acc: 44.55471124620061

Eval:
loss: tensor(3.5869, grad_fn=<DivBackward0>)    top1 acc: 21.835866261398177     top3 acc: 50.43465045592705

Epoch 4 :
Train:
loss: tensor(3.4853, grad_fn=<DivBackward0>)    top1 acc: 26.553951367781156     top3 acc: 54.60182370820669

Eval:
loss: tensor(3.3759, grad_fn=<DivBackward0>)    top1 acc: 28.857142857142858     top3 acc: 58.483282674772035

Epoch 5 :
Train:
loss: tensor(3

In [34]:
def cnn_pretrained_test(n_hidden,optim,learning_rate):
  gpu = -1 # default
  
  lr = learning_rate
  
  batch_size = 32 # default
  #ckpt_path = args.ckpt
  ckpt_path = './ckpt/cnn'
  
  n_epochs = 20 # default
 
  opt_str = optim

  ckpt_path = '%s/%s' % (ckpt_path, opt_str)

  if ckpt_path is not None:
    if not(os.path.exists(ckpt_path)):
      os.makedirs(ckpt_path)

  if gpu == -1:
    DEVICE = 'cpu'
  elif torch.cuda.is_available():
    DEVICE = gpu

  #model = CnnClassifier(n_hidden).to(DEVICE)
  # For the task 3.2.1, i need the Cnn_Scratch_Classifier model
  model = Cnn_Pretrained_Classifier(n_hidden)

  train_dataset, test_dataset = get_datasets()

  # Here I need to split the train_dataset in to train and eval with the ratio 4:1. I determine to set the last 20% as validation  dataset

  #n_samples = len(train_dataset)
  #eval_start = 0.8
  #split = int(np.floor(eval_start * n_samples))
  #indices = list(range(n_samples))
  #train_indices, validation_indices = indices[:split], indices[split:]

  dataloaders = {
    'train':  DataLoader(train_dataset,batch_size=batch_size, drop_last=False, shuffle=True),
    'eval':   DataLoader(test_dataset,batch_size=batch_size, drop_last=False, shuffle=True)
  }

  if opt_str == 'adam':
    opt_class = Adam
  elif opt_str == 'sgd':
    opt_class = SGD

  optimizer = opt_class(model.parameters(), lr=lr)
  writer = SummaryWriter('./logs/cnn_pretrained/test_phase_%s_%s_%s' % (opt_str,n_hidden,lr))

  train_for_test(model, dataloaders, optimizer, writer, n_epochs, ckpt_path, DEVICE)

  ########################################
  #the above part is the same as the train_main function
  # Now i am gonna to use the trained model to do the test_dataset
  loss =0.
  acc_1= 0.
  acc_3= 0. 
  dataloaders_test = DataLoader(test_dataset,batch_size=batch_size, drop_last=False, shuffle=True)  
  
  for i, batch in enumerate(dataloaders_test, 1):
      in_data, labels = batch 
      logits = model(in_data)
      batch_loss = model.loss(logits, labels)
      batch_acc_1 = model.top1_accuracy(logits, labels)
      batch_acc_3 = model.top3_accuracy(logits, labels)

      loss += batch_loss
      acc_1 += batch_acc_1
      acc_3 += batch_acc_3
    
  loss /= i
  acc_1 /= i
  acc_3 /= i

 ###########
  print('Test Phase:')
  print('test loss: %.4f acc1: %.4f acc3: %.4f' % (loss,acc_1,acc_3))



In [7]:
cnn_pretrained_test(64,'sgd',0.1)

Epoch 1 :
Train:
top1 acc: 29.83161094224924     top3 acc: 53.06869300911854

Test:
top1 acc: 50.5752427184466     top3 acc: 77.81796116504854

Epoch 2 :
Train:
top1 acc: 56.37629179331307     top3 acc: 81.87173252279635

Test:
top1 acc: 59.28883495145631     top3 acc: 83.18932038834951

Epoch 3 :
Train:
top1 acc: 61.564741641337385     top3 acc: 85.22066869300912

Test:
top1 acc: 61.70631067961165     top3 acc: 85.28398058252426

Epoch 4 :
Train:
top1 acc: 63.603039513677814     top3 acc: 86.44012158054711

Test:
top1 acc: 64.54611650485437     top3 acc: 86.62864077669903

Epoch 5 :
Train:
top1 acc: 64.79452887537994     top3 acc: 87.08632218844984

Test:
top1 acc: 65.10436893203884     top3 acc: 87.3252427184466

Epoch 6 :
Train:
top1 acc: 65.52158054711246     top3 acc: 87.54589665653495

Test:
top1 acc: 65.80097087378641     top3 acc: 87.17718446601941

Epoch 7 :
Train:
top1 acc: 66.06079027355624     top3 acc: 87.80364741641337

Test:
top1 acc: 66.49271844660194     top3 acc: 87.8

In [ ]:
def cnn_reconstructor_main(optim,learning_rate):
  
  gpu = -1 # default
  #lr = args.lr
  lr = learning_rate
  #batch_size = args.batch
  batch_size = 32 # default
  #ckpt_path = args.ckpt
  ckpt_path = './ckpt/cnn'
  #n_epochs = args.epoch
  n_epochs = 10 # default
  #opt_str = args.optim
  opt_str = optim

  ckpt_path = '%s/%s' % (ckpt_path, opt_str)

  if ckpt_path is not None:
    if not(os.path.exists(ckpt_path)):
      os.makedirs(ckpt_path)

  if gpu == -1:
    DEVICE = 'cpu'
  elif torch.cuda.is_available():
    DEVICE = gpu

  #model = CnnClassifier(n_hidden).to(DEVICE)
  # For the task 3.2.1, i need the Cnn_Scratch_Classifier model
  model = Cnn_Reconstructor()

  train_dataset, test_dataset = get_datasets()

  # Here I need to split the train_dataset in to train and eval with the ratio 4:1. I determine to set the last 20% as validation  dataset

  n_samples = len(train_dataset)
  eval_start = 0.8
  split = int(np.floor(eval_start * n_samples))
  indices = list(range(n_samples))
  train_indices, validation_indices = indices[:split], indices[split:]

  dataloaders = {
    'train':  DataLoader(Subset(train_dataset, indices=train_indices),batch_size=batch_size, drop_last=False, shuffle=True),
    'eval':   DataLoader(Subset(train_dataset, indices=validation_indices),batch_size=batch_size, drop_last=False, shuffle=True)
  }

  if opt_str == 'adam':
    opt_class = Adam
  elif opt_str == 'sgd':
    opt_class = SGD

  optimizer = opt_class(model.parameters(), lr=lr)
  writer = SummaryWriter('./logs/cnn_reconstructor/%s_%s' % (opt_str,lr))

  train_reconstructor(model, dataloaders, optimizer, writer, n_epochs, ckpt_path, DEVICE)













In [25]:
cnn_reconstructor_main('adam',0.001)

Epoch 1 :
Train:
loss: tensor(56.0705, grad_fn=<DivBackward0>)

Eval:
loss: tensor(35.5156, grad_fn=<DivBackward0>)

Epoch 2 :
Train:
loss: tensor(32.3901, grad_fn=<DivBackward0>)

Eval:
loss: tensor(30.1923, grad_fn=<DivBackward0>)

Epoch 3 :
Train:
loss: tensor(29.0210, grad_fn=<DivBackward0>)

Eval:
loss: tensor(28.2592, grad_fn=<DivBackward0>)

Epoch 4 :
Train:
loss: tensor(27.3596, grad_fn=<DivBackward0>)

Eval:
loss: tensor(26.9030, grad_fn=<DivBackward0>)

Epoch 5 :
Train:
loss: tensor(26.3834, grad_fn=<DivBackward0>)

Eval:
loss: tensor(26.3067, grad_fn=<DivBackward0>)

Epoch 6 :
Train:
loss: tensor(25.6910, grad_fn=<DivBackward0>)

Eval:
loss: tensor(25.4131, grad_fn=<DivBackward0>)

Epoch 7 :
Train:
loss: tensor(25.0006, grad_fn=<DivBackward0>)

Eval:
loss: tensor(24.8564, grad_fn=<DivBackward0>)

Epoch 8 :
Train:
loss: tensor(24.5487, grad_fn=<DivBackward0>)

Eval:
loss: tensor(24.6418, grad_fn=<DivBackward0>)

Epoch 9 :
Train:
loss: tensor(24.1936, grad_fn=<DivBackward0>)



In [24]:
cnn_reconstructor_main('sgd',0.01)

Epoch 1 :
Train:
loss: tensor(65.7541, grad_fn=<DivBackward0>)

Eval:
loss: tensor(48.6323, grad_fn=<DivBackward0>)

Epoch 2 :
Train:
loss: tensor(36.0014, grad_fn=<DivBackward0>)

Eval:
loss: tensor(30.8737, grad_fn=<DivBackward0>)

Epoch 3 :
Train:
loss: tensor(27.3142, grad_fn=<DivBackward0>)

Eval:
loss: tensor(26.7220, grad_fn=<DivBackward0>)

Epoch 4 :
Train:
loss: tensor(25.2783, grad_fn=<DivBackward0>)

Eval:
loss: tensor(23.5679, grad_fn=<DivBackward0>)

Epoch 5 :
Train:
loss: tensor(24.1115, grad_fn=<DivBackward0>)

Eval:
loss: tensor(22.6146, grad_fn=<DivBackward0>)

Epoch 6 :
Train:


KeyboardInterrupt: 

In [142]:
cnn_reconstructor_main('sgd',0.1)

Epoch 1 :
Train:
loss: tensor(111.1069, grad_fn=<DivBackward0>)

Eval:
loss: tensor(110.9498, grad_fn=<DivBackward0>)

Epoch 2 :
Train:
loss: tensor(111.0610, grad_fn=<DivBackward0>)

Eval:
loss: tensor(110.9495, grad_fn=<DivBackward0>)

Epoch 3 :
Train:
loss: tensor(111.0605, grad_fn=<DivBackward0>)

Eval:
loss: tensor(110.9490, grad_fn=<DivBackward0>)

Epoch 4 :
Train:
loss: tensor(111.0601, grad_fn=<DivBackward0>)

Eval:
loss: tensor(110.9484, grad_fn=<DivBackward0>)

Epoch 5 :
Train:
loss: tensor(111.0590, grad_fn=<DivBackward0>)

Eval:
loss: tensor(110.9469, grad_fn=<DivBackward0>)

Epoch 6 :
Train:
loss: tensor(111.0565, grad_fn=<DivBackward0>)

Eval:
loss: tensor(110.9421, grad_fn=<DivBackward0>)

Epoch 7 :
Train:
loss: tensor(108.7981, grad_fn=<DivBackward0>)

Eval:
loss: tensor(91.9122, grad_fn=<DivBackward0>)

Epoch 8 :
Train:
loss: tensor(95.5920, grad_fn=<DivBackward0>)

Eval:
loss: tensor(87.6523, grad_fn=<DivBackward0>)

Epoch 9 :
Train:
loss: tensor(94.9898, grad_fn=<Div

In [20]:
def cnn_reconstructor_test(optim,learning_rate):
  gpu = -1 # default
  
  lr = learning_rate
  
  batch_size = 32 # default
  #ckpt_path = args.ckpt
  ckpt_path = './ckpt/cnn'
  
  n_epochs = 10 # default
 
  opt_str = optim

  ckpt_path = '%s/%s' % (ckpt_path, opt_str)

  if ckpt_path is not None:
    if not(os.path.exists(ckpt_path)):
      os.makedirs(ckpt_path)

  if gpu == -1:
    DEVICE = 'cpu'
  elif torch.cuda.is_available():
    DEVICE = gpu

  #model = CnnClassifier(n_hidden).to(DEVICE)
  # For the task 3.2.1, i need the Cnn_Scratch_Classifier model
  model = Cnn_Reconstructor()

  train_dataset, test_dataset = get_datasets()


  dataloaders = {
    'train':  DataLoader(train_dataset,batch_size=batch_size, drop_last=False, shuffle=True),
    'eval':   DataLoader(test_dataset,batch_size=batch_size, drop_last=False, shuffle=True)
  }

  if opt_str == 'adam':
    opt_class = Adam
  elif opt_str == 'sgd':
    opt_class = SGD

  optimizer = opt_class(model.parameters(), lr=lr)
  writer = SummaryWriter('./logs/cnn_reconstructor/test_%s_%s' % (opt_str,lr))

  train_reconstructor(model, dataloaders, optimizer, writer, n_epochs, ckpt_path, DEVICE)

  ########################################
  #the above part is the same as the train_main function
  # Now i am gonna to use the trained model to do the test_dataset
  loss =0.
  acc_1= 0.
  acc_3= 0. 
  dataloaders_test = DataLoader(test_dataset,batch_size=batch_size, drop_last=False, shuffle=True)  
  
  for i, batch in enumerate(dataloaders_test, 1):
      in_data, labels = batch 
      logits = model(in_data)
      batch_loss = model.loss(logits, in_data)
      loss += batch_loss
        
      if len(logits.size()) == 2: # when it is flattened, reshape it
          logits = logits.view(-1, 1, 28, 28)

      img_grid = make_grid(logits.to('cpu'))

      writer.add_image('%s/test_final_reconstruction' % model.__class__.__name__, img_grid, 10)


    
  loss /= i
 ###########
  print('Test Phase:')
  print('test loss: %.4f ' % loss)
    
 # if len(logits.size()) == 2: # when it is flattened, reshape it
 #         logits = logits.view(-1, 1, 28, 28)

 # img_grid = make_grid(logits.to('cpu'))

 # writer.add_image('%s/test_reconstruction' % model.__class__.__name__, img_grid, 1)


    




In [21]:
cnn_reconstructor_test('sgd',0.01)

Epoch 1 :
Train:
loss: tensor(60.4159, grad_fn=<DivBackward0>)

Eval:
loss: tensor(43.0084, grad_fn=<DivBackward0>)

Epoch 2 :
Train:
loss: tensor(33.4852, grad_fn=<DivBackward0>)

Eval:
loss: tensor(27.2875, grad_fn=<DivBackward0>)

Epoch 3 :
Train:
loss: tensor(25.6187, grad_fn=<DivBackward0>)

Eval:
loss: tensor(23.7592, grad_fn=<DivBackward0>)

Epoch 4 :
Train:
loss: tensor(23.8127, grad_fn=<DivBackward0>)

Eval:
loss: tensor(22.3277, grad_fn=<DivBackward0>)

Epoch 5 :
Train:
loss: tensor(23.0043, grad_fn=<DivBackward0>)

Eval:
loss: tensor(22.4310, grad_fn=<DivBackward0>)

Epoch 6 :
Train:
loss: tensor(22.5741, grad_fn=<DivBackward0>)

Eval:
loss: tensor(22.4024, grad_fn=<DivBackward0>)

Epoch 7 :
Train:
loss: tensor(22.2219, grad_fn=<DivBackward0>)

Eval:
loss: tensor(21.3158, grad_fn=<DivBackward0>)

Epoch 8 :
Train:
loss: tensor(21.8545, grad_fn=<DivBackward0>)

Eval:
loss: tensor(21.1747, grad_fn=<DivBackward0>)

Epoch 9 :
Train:
loss: tensor(21.6566, grad_fn=<DivBackward0>)

